In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pandas import read_parquet

In [4]:
df=pd.read_csv('Automated_Traffic_Volume_Counts.csv')


In [5]:
df.head(5)

,RequestID,Boro,Yr,M,D,HH,MM,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction
0,26511,Manhattan,2018,3,3,0,0,5,70896,POINT (1001087.029976143 245331.74467825014),WEST 167 STREET,Audubon Avenue,8 Avenue Line,WB
1,26511,Manhattan,2018,3,3,0,15,5,70896,POINT (1001087.029976143 245331.74467825014),WEST 167 STREET,Audubon Avenue,8 Avenue Line,WB
2,26511,Manhattan,2018,3,3,0,30,6,70896,POINT (1001087.029976143 245331.74467825014),WEST 167 STREET,Audubon Avenue,8 Avenue Line,WB
3,26511,Manhattan,2018,3,3,0,45,10,70896,POINT (1001087.029976143 245331.74467825014),WEST 167 STREET,Audubon Avenue,8 Avenue Line,WB
4,26511,Manhattan,2018,3,3,1,0,2,70896,POINT (1001087.029976143 245331.74467825014),WEST 167 STREET,Audubon Avenue,8 Avenue Line,WB


In [14]:
unique_column=df['SegmentID'].nunique()
print(unique_column)

371


In [6]:
select_column=['Yr','M','D','HH','MM','Vol','SegmentID','Direction']
df[select_column].to_parquet('volume_data')

In [11]:
df=pd.read_parquet('volume_data')

datetime_format = "%d %m %Y %H %M"


df['datetime'] = pd.to_datetime(df[['D', 'M', 'Yr', 'HH', 'MM']].astype(str).apply(' '.join, axis=1), format=datetime_format)


print(df['datetime'].head(2))


0   2018-03-03 00:00:00
1   2018-03-03 00:15:00
Name: datetime, dtype: datetime64[ns]


In [13]:
drop_column=['D', 'M', 'Yr', 'HH', 'MM']
df=df.drop(drop_column,axis=1)
df.to_parquet('volume_data')

In [189]:
df=pd.read_parquet('volume_data')
df.head(5)

,Vol,SegmentID,Direction,datetime
0,5,70896,WB,2018-03-03 00:00:00
1,5,70896,WB,2018-03-03 00:15:00
2,6,70896,WB,2018-03-03 00:30:00
3,10,70896,WB,2018-03-03 00:45:00
4,2,70896,WB,2018-03-03 01:00:00


In [15]:
null_values = df.isnull().sum()
print(null_values)

Vol          0
SegmentID    0
Direction    0
datetime     0
dtype: int64


In [191]:
df.shape

(497668, 4)

In [24]:
unique_rows = df.drop_duplicates(subset='SegmentID', keep='first')
# print(unique_rows.count())

select_column=['SegmentID','WktGeom']
print(unique_rows[select_column])

        SegmentID                                       WktGeom
0           70896  POINT (1001087.029976143 245331.74467825014)
1248        33615   POINT (985031.393485933 217039.22504542628)
1249        33411  POINT (982877.3474610402 213144.49845334244)
1250        33968    POINT (987795.1707192899 216449.397035686)
1251        33850  POINT (986902.8082711459 216696.10890837928)
...           ...                                           ...
483948      38363    POINT (995544.6304203826 233563.592012279)
483949      38802   POINT (998686.8133056197 239356.0580135274)
483950      38689    POINT (997828.5077050285 238642.142675619)
496038     146126  POINT (993050.8054429016 212926.56866003707)
497483     194467  POINT (992165.1300347954 210393.60518822112)

[371 rows x 2 columns]


In [28]:
from shapely.wkt import loads
from pyproj import CRS, Transformer

# 定义源坐标系（EPSG 代码或 Proj4 字符串）
source_crs = CRS('EPSG:2263')  # 示例：NAD83 / New York Long Island (ftUS)

# 定义目标坐标系（EPSG 代码或 Proj4 字符串）
target_crs = CRS('EPSG:4326')  # 示例：WGS84 经纬度坐标系

# 创建坐标转换器
transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)

# 创建空的数组来存储转换后的坐标
coordinates = []

# 遍历 unique_rows 中的每个几何数据
for geom_str in unique_rows['WktGeom']:
    # 从 WKT 格式加载几何对象
    geom = loads(geom_str)

    # 提取点的坐标
    x, y = geom.x, geom.y

    # 进行坐标转换
    lon, lat = transformer.transform(x, y)

    # 将转换后的坐标作为元组添加到数组中
    coordinates.append((lat, lon))

# 将转换后的坐标数组赋值给 unique_rows 的新列 Coordinates
unique_rows['Coordinates'] = coordinates

# 打印包含转换后坐标的 unique_rows
print(unique_rows['Coordinates'])


0          (40.84003811194584, -73.93915023590672)
1248        (40.762398908462565, -73.997179301792)
1249       (40.75170880559713, -74.00495424814217)
1250      (40.760779305883766, -73.98720284497887)
1251      (40.761456779413706, -73.99042394665165)
                            ...                   
483948     (40.80774670346572, -73.95920051630279)
483949     (40.82364082223198, -73.94783758212786)
483950    (40.821682688209705, -73.95094021573594)
496038     (40.75110637030952, -73.96823596692832)
497483     (40.74415485986301, -73.97143554419223)
Name: Coordinates, Length: 371, dtype: object


C:\Users\xuhuian\AppData\Local\Temp\ipykernel_6076\525333213.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_rows['Coordinates'] = coordinates


In [29]:
unique_rows.head(2)

,RequestID,Boro,Yr,M,D,HH,MM,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction,Coordinates
0,26511,Manhattan,2018,3,3,0,0,5,70896,POINT (1001087.029976143 245331.74467825014),WEST 167 STREET,Audubon Avenue,8 Avenue Line,WB,"(40.84003811194584, -73.93915023590672)"
1248,26515,Manhattan,2018,3,3,0,0,4,33615,POINT (985031.393485933 217039.22504542628),11 AVENUE,West 44 Street,West 45 Street,NB,"(40.762398908462565, -73.997179301792)"


In [30]:
select_column=['SegmentID','Coordinates']
unique_rows[select_column].to_csv('volume_SegmentID-Geolocation')

In [42]:
df_new=pd.read_csv('volume_SegmentID-Geolocation.csv')
df_new.shape

(371, 4)

In [43]:
df_new.dtypes

Unnamed: 0.1     int64
Unnamed: 0       int64
SegmentID        int64
Coordinates     object
dtype: object

In [47]:
df_new=df_new.drop('Unnamed: 0.1',axis=1)

In [48]:
df_new=df_new.drop('Unnamed: 0',axis=1)

In [49]:
df_new.head(2)

,SegmentID,Coordinates
0,70896,"(40.84003811194584, -73.93915023590672)"
1,33615,"(40.762398908462565, -73.997179301792)"


In [50]:
df_new.to_csv('volume_SegmentID-Geolocation.csv')

In [51]:
df_swap=pd.read_csv('volume_SegmentID-Geolocation.csv')

In [56]:
# 定义交换坐标顺序的函数
def swap_coordinates(coordinates_str):
    lat, lon = coordinates_str.strip('()').split(', ')
    return f'({lon}, {lat})'

# 在DataFrame中应用交换坐标顺序的函数
df_swap['Coordinates'] = df_swap['Coordinates'].apply(lambda x: swap_coordinates(x) if pd.notnull(x) else x)

# 打印交换后的DataFrame
print(df_swap)


     Unnamed: 0  SegmentID                               Coordinates
0             0      70896   (-73.93915023590672, 40.84003811194584)
1             1      33615    (-73.997179301792, 40.762398908462565)
2             2      33411   (-74.00495424814217, 40.75170880559713)
3             3      33968  (-73.98720284497887, 40.760779305883766)
4             4      33850  (-73.99042394665165, 40.761456779413706)
..          ...        ...                                       ...
366         366      38363   (-73.95920051630279, 40.80774670346572)
367         367      38802   (-73.94783758212786, 40.82364082223198)
368         368      38689  (-73.95094021573594, 40.821682688209705)
369         369     146126   (-73.96823596692832, 40.75110637030952)
370         370     194467   (-73.97143554419223, 40.74415485986301)

[371 rows x 3 columns]


In [57]:
df_swap=df_swap.drop('Unnamed: 0',axis=1)

In [58]:
df_swap.head(2)

,SegmentID,Coordinates
0,70896,"(-73.93915023590672, 40.84003811194584)"
1,33615,"(-73.997179301792, 40.762398908462565)"


In [59]:
df_swap.to_csv('volume_SegmentID-Geolocation.csv')